In [1]:
from sagemaker import Session

sess = Session()

model_data = sess.upload_data(
    path="/Users/iminjun/Documents/git/yolo/model.tar.gz", bucket='sagemaker-yolo-midpare', key_prefix="model"
)

print(model_data)

s3://sagemaker-yolo-midpare/model/model.tar.gz


In [ ]:
from sagemaker.pytorch import PyTorchModel

role = 'arn:aws:iam::972511216533:role/service-role/AmazonSageMaker-ExecutionRole-20240507T105157'

model = PyTorchModel(
    entry_point="inference.py",
    source_dir="code",
    role=role,
    model_data=model_data,
    framework_version="1.9.0",
    py_version="py38",
)

In [25]:
from sagemaker.deserializers import JSONDeserializer

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    deserializer=JSONDeserializer(),
)

-------!

In [4]:
# from sagemaker.predictor import Predictor
# from sagemaker.local import LocalSession

# endpoint_name = model.endpoint_name
# local_sess = LocalSession()

# predictor = Predictor(
#     endpoint_name=endpoint_name, 
#     sagemaker_session=local_sess,
# )

In [22]:
import boto3, cv2, json, random, io
import numpy as np

runtime_client = boto3.client('sagemaker-runtime')


orig_image = cv2.imread('/Users/iminjun/Documents/git/yolo/assests/bus.jpg')
# orig_image = cv2.imread('/Users/iminjun/Downloads/IMG_7502.HEIC')
payload = cv2.imencode('.jpg', orig_image)[1].tobytes()

response = runtime_client.invoke_endpoint(
    EndpointName='pytorch-inference-2024-05-23-14-19-25-097',
    ContentType='image/jpeg',
    Accept='application/json',
    Body=payload
)

output = json.loads(response["Body"].read().decode('utf-8'))

print(json.dumps(output, sort_keys=True, indent=4))


KeyboardInterrupt: 